# Обучение агента в Unity ML-Agents с экспортом ONNX

Этот ноутбук обучает агента из Unity-среды (`MyAgent?team=0`) с 6 наблюдениями, 2 непрерывными действиями и 1 дискретным действием. Используем PPO из `stable-baselines3` для обучения и экспортируем модель в ONNX для Unity.

## Зависимости
- Установите необходимые библиотеки:
  ```bash
  pip install mlagents stable-baselines3 torch onnx numpy
  ```
- Убедитесь, что путь к `UnityEnvironment.exe` правильный.
- Среда Unity должна быть собрана с `Behavior Name: MyAgent?team=0`.

In [2]:
!pip install stable-baselines3 onnx

  Using cached gymnasium-1.0.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
   ---------------------------------------- 0.0/958.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/958.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/958.1 kB ? eta -:--:--
   ---------- ----------------------------- 262.1/958.1 kB ? eta -:--:--
   ---------- ----------------------------- 262.1/958.1 kB ? eta -:--:--
   ---------- ----------------------------- 262.1/958.1 kB ? eta -:--:--
   ---------- ----------------------------- 262.1/958.1 kB ? eta -:--:--
   -------------------- ----------------- 524.3/958.1 kB 328.9 kB/s eta 0:00:02
   -------------------- ----------------- 524.3/958.1 kB 328.9 kB/s eta 0:00:02
   -------------------- ----------------- 524.3/958.1 kB 328.9 kB/s eta 0:00:02
   -------------------- ----------------- 524.3/958.1 kB 328.9 kB/s eta 0:00:02
   --------------------------

In [3]:
# Импорт библиотек
import os
import numpy as np
import torch
import torch.nn as nn
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from mlagents_envs.base_env import ActionTuple
from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.env_checker import check_env
from gym import spaces
import gym

# Путь к среде Unity
env_path = os.path.join(os.getcwd(), r'N:\MyRL\My_First_NPC\MyfirstMPC\UnityEnvironment.exe')  # Укажите свой путь

# Настройка канала для ускорения симуляции
engine_channel = EngineConfigurationChannel()
engine_channel.set_configuration_parameters(time_scale=50.0, quality_level=0)

# Инициализация среды
env = UnityEnvironment(file_name=env_path, side_channels=[engine_channel])
env.reset()

# Получение имени поведения
behavior_name = list(env.behavior_specs.keys())[0]
print(f'Behavior Name: {behavior_name}')
spec = env.behavior_specs[behavior_name]

# Проверка спецификации
print(f'Observation size: {spec.observation_specs[0].shape[0]}')
print(f'Continuous action size: {spec.action_spec.continuous_size}')
print(f'Discrete action branches: {spec.action_spec.discrete_branches}')

Behavior Name: MyAgent?team=0
Observation size: 6
Continuous action size: 2
Discrete action branches: (1,)


## Создание кастомной нейросети

Определяем нейросеть с двумя выходами: для непрерывных действий (2) и дискретного действия (1). Используем `BaseFeaturesExtractor` для совместимости с `stable-baselines3`.

In [4]:
# Кастомная нейросеть
class CustomActorCriticNet(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=128):
        super(CustomActorCriticNet, self).__init__(observation_space, features_dim)
        self.fc1 = nn.Linear(observation_space.shape[0], 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, features_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Определяем политику для PPO
policy_kwargs = dict(
    features_extractor_class=CustomActorCriticNet,
    features_extractor_kwargs=dict(features_dim=128),
    net_arch=[dict(pi=[64, 32], vf=[64, 32])]  # Архитектура для актора и критика
)

## Обёртка среды для Gym

Создаём обёртку Gym, чтобы `stable-baselines3` мог работать с Unity ML-Agents.

In [5]:
class UnityGymWrapper(gym.Env):
    def __init__(self, unity_env, behavior_name, spec):
        super(UnityGymWrapper, self).__init__()
        self.env = unity_env
        self.behavior_name = behavior_name
        self.spec = spec
        
        # Пространства наблюдений и действий
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(spec.observation_specs[0].shape[0],), dtype=np.float32
        )
        self.action_space = spaces.MultiDiscrete(spec.action_spec.discrete_branches) if spec.action_spec.continuous_size == 0 else \
                            spaces.Box(low=-1.0, high=1.0, shape=(spec.action_spec.continuous_size,), dtype=np.float32)
        
        # Если есть и непрерывные, и дискретные действия
        if spec.action_spec.continuous_size > 0 and spec.action_spec.discrete_size > 0:
            self.action_space = spaces.Dict({
                'continuous': spaces.Box(low=-1.0, high=1.0, shape=(spec.action_spec.continuous_size,), dtype=np.float32),
                'discrete': spaces.MultiDiscrete(spec.action_spec.discrete_branches)
            })

    def reset(self):
        self.env.reset()
        decision_steps, terminal_steps = self.env.get_steps(self.behavior_name)
        return decision_steps.obs[0][0]  # Возвращаем первое наблюдение

    def step(self, action):
        # Создаём ActionTuple
        action_tuple = ActionTuple()
        if isinstance(self.action_space, spaces.Dict):
            action_tuple.add_continuous(action['continuous'])
            action_tuple.add_discrete(action['discrete'].reshape(-1, len(self.spec.action_spec.discrete_branches)))
        else:
            if self.spec.action_spec.continuous_size > 0:
                action_tuple.add_continuous(action)
            else:
                action_tuple.add_discrete(action.reshape(-1, len(self.spec.action_spec.discrete_branches)))

        self.env.set_actions(self.behavior_name, action_tuple)
        self.env.step()

        decision_steps, terminal_steps = self.env.get_steps(self.behavior_name)
        done = len(terminal_steps) > 0
        reward = terminal_steps.reward[0] if done else decision_steps.reward[0]
        obs = terminal_steps.obs[0][0] if done else decision_steps.obs[0][0]
        info = {}

        return obs, reward, done, info

    def close(self):
        self.env.close()

# Создаём обёртку
gym_env = UnityGymWrapper(env, behavior_name, spec)
check_env(gym_env)  # Проверяем совместимость с Gym

AssertionError: Your environment must inherit from the gymnasium.Env class cf. https://gymnasium.farama.org/api/env/

## Обучение модели

Используем PPO для обучения. Гиперпараметры подобраны для стабильного обучения.

In [ ]:
# Инициализация PPO
model = PPO(
    'MlpPolicy',
    gym_env,
    policy_kwargs=policy_kwargs,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.05,
    ent_coef=0.01,
    verbose=1
)

# Обучение на 100,000 шагов
model.learn(total_timesteps=100000)

# Сохранение модели
model.save('ppo_myagent')

## Экспорт модели в ONNX

Экспортируем модель в формат ONNX, совместимый с Unity Barracuda. Учитываем имена входов/выходов и динамический батч.

In [ ]:
# Получаем политику из модели
policy = model.policy

# Создаём фиктивный вход
dummy_input = torch.randn(1, spec.observation_specs[0].shape[0]).to(policy.device)

# Экспорт в ONNX
torch.onnx.export(
    policy,
    dummy_input,
    'trained_myagent.onnx',
    input_names=['obs_0'],
    output_names=['continuous_actions', 'discrete_actions'],
    dynamic_axes={
        'obs_0': {0: 'batch_size'},
        'continuous_actions': {0: 'batch_size'},
        'discrete_actions': {0: 'batch_size'}
    },
    opset_version=9,
    verbose=False
)

print('Модель успешно сохранена: trained_myagent.onnx')

# Проверка существования файла
print('Файл существует:', os.path.exists('trained_myagent.onnx'))

## Тестирование модели

Проверяем, как обученная модель работает в среде.

In [ ]:
# Тестирование
obs = gym_env.reset()
for _ in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = gym_env.step(action)
    if done:
        obs = gym_env.reset()
        print('Эпизод завершён')

# Закрытие среды
gym_env.close()